partially generated using copilot
this does the following:
- goes through each folder in the path,

- grabs the modify time of this file:
[folder]/network_files/[network cutoff folder]/mix/"mix_c0.30.network"
this is the network file that is generated during distance calculation

- grabs the **creation** time of this file:
[folder]/html_content/networks/[network cutoff folder]/mix/bs_networks.js
this file is created during the gcf calling step.
bigscape v1 crashes during the creation of this file in large datasets

- grabs the last modified time of this file:
[folder]/logs/runtimes.txt
this is the last file to be touched during the execution of bigscape v1

- calculates the difference between the network file mtime and runtimes.txt file ctime
this is the total execution time of the gcf calling step
- calculates the difference between the creation time of the js file and the runtimes.txt file
this is the portion that is missing in the v1 crash

- after collecting all the data, calculates the missing time as a projection

In [3]:
from datetime import timedelta
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set_theme(style="white")

COLORS = {
    "bigscape_blue": "#52A3A3",
    "dark_blue": "#0E75BB",
    "orange_i_found_on_bigscape_image": "#F7931E",
    "antismash_red": "#AA0000",
    "bigslice_grey": "#515154",
}

mpl.rcParams["svg.fonttype"] = "none"

In [4]:
# convert all to numpy arrays
post_crash_estimate_array = pd.read_csv("source_data/bigscape_v1_crash_timings.csv")

# fit a polynomial to the data
p = np.polyfit(
    post_crash_estimate_array[:, 0],
    post_crash_estimate_array[:, 1],
    2,
)

np.polyfit_coeffs = p


# calculate R^2
# sum squares of the residuals
residuals = post_crash_estimate_array[:, 1] - np.polyval(np.polyfit_coeffs, post_crash_estimate_array[:, 0])

ss_res = np.sum(residuals**2)
# total sum of squares
ss_tot = np.sum((post_crash_estimate_array[:, 1] - np.mean(post_crash_estimate_array[:, 1]))**2)
# R^2
r_squared = 1 - (ss_res / ss_tot)
print(f"R^2: {r_squared:.4f}")


# get the estimated post crash time for 50k samples
estimate = np.polyval(np.polyfit_coeffs, 50000)

print(
    f"Estimated missing run time for 50k samples: {estimate} seconds"
)

# convert the estimate to a timedelta
estimate_time_span = timedelta(seconds=estimate)

# add the estimate to the pre crash time
estimate_end_time = pre_crash_50k + estimate_time_span

# calculate the total run time by subtracting the estimated end time from the start time
estimate_runtime = estimate_end_time - start_50k
print(f"Estimated runtime for 50k samples: {estimate_runtime.total_seconds():.0f} seconds")



# add the estimate to the data
post_crash_estimate_array = np.vstack(
    [post_crash_estimate_array, [50000, estimate]]
)

# scatterplot with plot of the fit
sns.scatterplot(
    x=post_crash_estimate_array[:, 0],
    y=post_crash_estimate_array[:, 1],
    label="Post crash data",
)
x_fit = np.linspace(0, 50000, 100)
y_fit = np.polyval(np.polyfit_coeffs, x_fit)
sns.lineplot(x=x_fit, y=y_fit, label="Fit", color=COLORS["bigscape_blue"])
plt.xlabel("# records")
plt.ylabel("Missing runtime (seconds)")
plt.title("Missing time estimation for Bigscape v1")
plt.legend()

# add text with the estimate
plt.text(
    0.05,
    0.95,
    f"R² = {r_squared:.4f}\n",
    transform=plt.gca().transAxes,
    fontsize=12,
    verticalalignment="top",
    bbox=dict(facecolor="white", alpha=0.5, edgecolor="none"),
)


plt.tight_layout()
plt.savefig("post_crash_estimation_v1.svg", bbox_inches="tight")

InvalidIndexError: (slice(None, None, None), 0)